# SpaceX Launch Dash

In [1]:
# !pip install jupyter-dash

In [2]:
# instead of import dash
# use this in JupyterLab
import pandas as pd
import jupyter_dash
import dash
from dash import html
# import dash_html_components as html
from dash import dcc
# import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# download dataset as spacex_launch_dash.csv
# !wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

In [4]:
# !wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"

In [5]:
# Read the data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [6]:
# Create a dash application
app = jupyter_dash.JupyterDash(__name__)
# instead of app = dash.Dash(__name__) in standard dash

# Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [7]:
# get launch sites and put into dictionary for constructor

launch_sites = spacex_df['Launch Site'].unique()

options_dict = [{'label': 'ALL',  'value': 'ALL'},]
for site in launch_sites:
    dict_launches= {'label': site,  'value': site}
    options_dict.append(dict_launches)

In [8]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                 dcc.Dropdown(id='site-dropdown',
                                    options= options_dict,
                                    value='ALL',
                                    placeholder="Select a Launch Site here",
                                    searchable=True
                                    ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),


                                 # return the outcomes piechart for a selected site
                                html.P("Payload range (Kg):"),
                                # Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0',
                                        100: '100'},
                                    value=[min_payload, max_payload]),
                                
                                # Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [9]:
class_names = ['Failure','Success']

In [10]:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
            Input(component_id='site-dropdown', component_property='value'))
        
def get_pie_chart(l_site):
    
    filtered_df= spacex_df[['Launch Site','class']]
  
    if l_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title=l_site)
        return fig
    else: 
        count_values =[len(filtered_df.loc[(filtered_df['Launch Site'] == l_site) & (filtered_df['class']==0)]), len(filtered_df.loc[(filtered_df['Launch Site'] == l_site) & (filtered_df['class']==1)])]
        fig = px.pie(filtered_df.loc[filtered_df['Launch Site'] == l_site], values= count_values,
        names = class_names,
        title= l_site)
        return fig


In [11]:
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value'),
                Input(component_id='payload-slider', component_property='value'))
def get_scatter_chart(l_site,value):
    
    filtered_df2= spacex_df[['Launch Site','class','Payload Mass (kg)','Booster Version Category']]

    if l_site == 'ALL':
        fig = px.scatter(spacex_df.loc[spacex_df['Payload Mass (kg)'].between(value[0],value[1])], x = 'Payload Mass (kg)', y = 'class' ,color="Booster Version Category")
        return fig
    else:
                
        fig = px.scatter(spacex_df.loc[(spacex_df['Launch Site'] == l_site) & (spacex_df['Payload Mass (kg)'].between(value[0],value[1]))], x = 'Payload Mass (kg)', y = 'class' ,color="Booster Version Category")
        return fig

In [12]:
# Run the app
if __name__ == '__main__':
    # app.run_server()
    app.run_server(mode="inline")